In [1]:
import requests
from bs4 import BeautifulSoup
import os
import re


GENIUS_API_TOKEN='YOUR_API_TOKEN_KEY' # Mettre votre propre clé API token à récupérer sur le site de genius.com


# Récupère l'objet de l'artiste depuis l'API Genius
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data=data, headers=headers)
    
    return response


# Récupère les URLs des artistes depuis l'objet artiste
def request_song_url(artist_name, song_cap):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()

        # Récupère l'objet chanson de l'artiste
        song_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                song_info.append(hit)
    
        # Récupère l'URL de la chanson de l'objet chanson
        for song in song_info:
            if (len(songs) < song_cap):
                url = song['result']['url']
                songs.append(url)

        if (len(songs) == song_cap):
            break
        else:
            page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))

    return songs


# un peu similaire à la fonction request_song_url sauf que l'on récupère toutes les URLs des chansons de l'artiste
def request_all_songs_url(artist_name):
    page = 1
    songs = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        
        song_info = json['response']['hits']
        for hit in song_info:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                url = hit['result']['url']
                songs.append(url)
    
        # On vérifie si il y a encore des chansons sinon on sort de la boucle
        if not song_info:
            break
            
        page += 1
        
    print('Found {} songs by {}'.format(len(songs), artist_name))

    return songs


# Fonction qui sert à éliminer des lignes superflues récurrentes
def len_header(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    try:
        element = html.find('ul', class_='LyricsHeader__DropdownContents-ejidji-4 fSmeGU').get_text(separator=',').split(",")
    except:
        element = []

    return len(element)+2


# Scrape les lyrics d'une chanson avec l'url donné
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', id='lyrics-root').get_text(separator='\n')
    lyrics_lines = lyrics.split('\n')
    # On utilise la fonction len_header pour supprimer les premières lignes superflues et on met -2 pour enlever les deux lignes superflues à la fin
    lyrics = '\n'.join(lyrics_lines[len_header(url):-2])
    # Supprime les identificateurs tels que "chorus", "verse", etc., en utilisant des expressions régulières
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    # On supprime les lignes vides
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])  

    # On retourne les lyrics avec un double saut de ligne pour bien séparer les chansons       
    return lyrics+"\n\n"


# On écrit les lyrics dans un fichier au nom de l'artiste
def write_lyrics_to_file(artist_name):
    f = open('lyrics/' + artist_name.lower() + '.txt', 'wb')
    urls = request_all_songs_url(artist_name)
    for url in urls:
        lyrics = scrape_song_lyrics(url)
        f.write(lyrics.encode("utf8"))
    f.close()
    num_lines = sum(1 for line in open('lyrics/' + artist_name.lower() + '.txt', 'rb'))
    print('Wrote {} lines to file from {} songs'.format(num_lines, len(urls)))

In [2]:
# On peut afficher les liens vers les pages à scraper
request_song_url('Dua Lipa', 10)

Found 10 songs by Dua Lipa


['https://genius.com/Dua-lipa-new-rules-lyrics',
 'https://genius.com/Martin-garrix-and-dua-lipa-scared-to-be-lonely-lyrics',
 'https://genius.com/Dua-lipa-dont-start-now-lyrics',
 'https://genius.com/Dua-lipa-idgaf-lyrics',
 'https://genius.com/Dua-lipa-levitating-lyrics',
 'https://genius.com/Calvin-harris-and-dua-lipa-one-kiss-lyrics',
 'https://genius.com/Dua-lipa-blow-your-mind-mwah-lyrics',
 'https://genius.com/Dua-lipa-break-my-heart-lyrics',
 'https://genius.com/Dua-lipa-and-blackpink-kiss-and-make-up-lyrics',
 'https://genius.com/Dua-lipa-be-the-one-lyrics']

In [3]:
# On peut afficher les lyrics pour voir si ça marche bien
print(scrape_song_lyrics('https://genius.com/Nekfeu-humanoide-lyrics'))

Est-ce que tu t'es d'jà fait rabaisser par celle que t'aimais secrètement ?
Gentille en privé mais, d'vant les gens, cruelle et légère
Est-ce que t'as d'jà tapé quelqu'un juste pour qu'on te respecte ?
Sans excuse, est-ce que t'as d'jà regretté au point d'ber-ger ?
Est-ce que tu t'es d'jà dit : "Faut qu'j'me secoue, j'ai plus 16 ans" ?
N'être plus qu'une âme perdue, errer et rôder des heures
Voir un mec s'faire racketter, appeler au s'cours et pisser l'sang
Dans le même wagon du RER où t'étais seul
Est-ce que t'as détourné l'regard ? Dégoûté face à ta propre lâcheté
Est-ce qu'après t'as fait des trucs de malade juste pour t'racheter ?
Est-ce pour ça qu't'as été violent quand la prof t'a jeté ?
Est-ce que tu t'es identifié au taf de Kourtrajmé ?
Est-ce que tu gardais, à l'époque
Des secrets d'famille lourds dont tu peux même pas parler à tes potes ?
Même pas deux balles pour un café, rêvais-tu d'sauter la serveuse ?
L'été, à Paris, dans les parcs, tu sortais la serviette
Est-ce que t'as

In [4]:
# Ici on a la façon de le faire sans boucle pour seulement avoir un artiste

# write_lyrics_to_file('NewJeans')

In [5]:
# On récupère les lyrics de l'artiste (ou des artistes) que l'on veut et on le stock dans un fichier txt à son nom
list_pop_artists = ["Dua Lipa", "The Weeknd", "Billie Eilish", "Harry Styles", "Ariana Grande", "Taylor Swift", "Justin Bieber", "Doja Cat"]

for artist in list_pop_artists:
    write_lyrics_to_file(artist)

Found 469 songs by Justin Bieber
Wrote 21661 lines to file from 469 songs
Found 383 songs by Doja Cat
Wrote 17769 lines to file from 383 songs


In [6]:
# Fusionner les fichiers txt
filenames = ["lyrics/ariana grande.txt", "lyrics/billie eilish.txt", "lyrics/doja cat.txt", "lyrics/dua lipa.txt", "lyrics/harry styles.txt", "lyrics/justin bieber.txt", "lyrics/taylor swift.txt", "lyrics/the weeknd.txt"]

# Ici j'appelle le fichier pop_lyrics car j'ai pris que des artistes pop
with open("lyrics/pop_lyrics.txt", "w", encoding="utf8") as new_file:
    for name in filenames:
        with open(name, encoding="utf8") as f:
            for line in f:
                new_file.write(line)

            new_file.write("\n")